<a href="https://colab.research.google.com/github/DCDPUAEM/DCDP/blob/main/03%20Machine%20Learning/notebooks/08-Comparacion-clasificadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparación de varios clasificadores

![alt text](https://drive.google.com/uc?id=1lcj0sqCwfqbV1DHGyYKsq6SNlkKpKEXY )

En esta notebook compararemos el rendimiento y tiempo de ejecución de varios algoritmos de clasificación.

* `KNeighborsClassifier`: Clasifica comparando una instancia de acuerdo a las etiquetas de los vecinos más cercanos en el espacio de features.

* `AdaBoostClassifier`: Es un meta-algoritmo que crea un ensamble de clasificadores con una técnica diferente al *bagging*.

* `BaggingClassifier`: Es un meta-algoritmo que crea un ensamble de clasificadores usando *bagging*.

* `GaussianNB` (Bayes Ingenuo Gaussiano): Se clasifican las instancias calculando la probabilidad de que cada instancia pertenezca a cada clase, dadas sus features, bajo la suposición de que las features siguen una distribución normal y son independientes entre sí dadas la clase (esta última es la suposición *naive*).

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import time
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=1001)

Corremos un entrenamiento por cada clasificador, midiendo el tiempo de entrenamiento y el accuracy en el conjunto de prueba.

In [ ]:
clfs = [DecisionTreeClassifier(),RandomForestClassifier(),
        SVC(),LogisticRegression(),
        GaussianNB(),KNeighborsClassifier(),
        AdaBoostClassifier(estimator=SVC(),algorithm='SAMME'),
        AdaBoostClassifier(estimator=GaussianNB(),algorithm='SAMME'),
        BaggingClassifier(estimator=SVC())]

names = [x.__class__.__name__ for x in clfs]  # Obtener el nombre de cada clasificador, como <string>
names[-3] += '_SVC'
names[-2] += '_GNB'
names[-1] += '_SVC'
times = []
accs = []

for clf in clfs:
    pl = Pipeline([('scl',StandardScaler()),
                    ('clf',clf)])
    start = time.time()
    pl.fit(X_train,y_train)
    end = time.time()
    times.append(end-start)
    y_pred = pl.predict(X_test)
    accs.append(accuracy_score(y_test,y_pred))

resultados_df = pd.DataFrame(data={'algoritmo': names,
                   'accuracy': accs,
                   'duración': times})

Corremos **5** entrenamiento por cada clasificador, midiendo el tiempo **promedio** de entrenamiento y el accuracy **promedio** en el conjunto de prueba.

⏰ Esta celda tarda alrededor de 3 minutos.

In [ ]:
clfs = [DecisionTreeClassifier(),RandomForestClassifier(),
        SVC(),LogisticRegression(),
        GaussianNB(),KNeighborsClassifier(),
        AdaBoostClassifier(estimator=SVC(),algorithm='SAMME'),
        AdaBoostClassifier(estimator=GaussianNB(),algorithm='SAMME'),
        BaggingClassifier(estimator=SVC())]

names = [x.__class__.__name__ for x in clfs]  # Obtener el nombre de cada clasificador, como <string>
names[-3] += '_SVC'
names[-2] += '_GNB'
names[-1] += '_SVC'
times = []
accs = []

for clf in clfs:
    times_clf = []
    accs_clf = []
    for k in range(5):
        pl = Pipeline([('scl',StandardScaler()),
                        ('clf',clf)])
        start = time.time()
        pl.fit(X_train,y_train)
        end = time.time()
        times_clf.append(end-start)
        y_pred = pl.predict(X_test)
        accs_clf.append(accuracy_score(y_test,y_pred))
    times.append(np.mean(times_clf))
    accs.append(np.mean(accs_clf))
    accs_clf.append(accuracy_score(y_test,y_pred))
resultados_df = pd.DataFrame(data={'algoritmo': names,
                   'accuracy': accs,
                   'duración': times})

## Rendimiento

In [ ]:
resultados_df.sort_values(by='accuracy',ascending=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
sns.barplot(x='algoritmo',y='accuracy',data=resultados_df)
plt.xticks(rotation=90)
plt.xlabel('Algoritmo')
plt.ylabel('Accuracy')
plt.show()

## Tiempo de ejecución

Ordenados de más rápido a más lento

In [ ]:
resultados_df.sort_values(by='duración',ascending=True)

In [ ]:
idxs = resultados_df.index.to_list()
idxs.remove(6)
idxs

plt.figure(figsize=(10,5))
sns.barplot(x='algoritmo',y='duración',data=resultados_df.loc[idxs])
plt.xticks(rotation=90)
plt.xlabel('Algoritmo')
plt.ylabel('Duración (s)')
plt.show()

A continuación se presenta la complejidad big-O de algunos algoritmos:

* Naive Bayes: $O(Nd)$
* k-NN: $O(1)$ (space complexity of training is $O(Nd)$ since the data needs to be stored, which also takes time).
* SVM: $O(N^2)$ or $O(N^3)$ depending on the kernel.
* DecisionTree: $O(n\log n)$

# Comparación de las fronteras de decisión: Ejemplo 3

In [ ]:
#@title Conjunto de datos
import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

X, y = make_blobs(n_samples=600,centers=2,random_state=31)

theta = np.pi/4 # Ángulo de rotación
R = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]]) # Matriz de rotación

Xr = np.transpose(R@np.transpose(X)) # Rotamos el dataset

idxs = np.where(y==1)[0]   # Obtenemos los índices donde y=1

Xr[idxs,:] = Xr[idxs,:] + np.array([-1,-2])

X1, y1 = Xr, y

plt.figure(figsize=(4,4))
plt.scatter(X1[:,0],X1[:,1],c=y1)
plt.axis('off')
plt.axis('equal')
plt.show()

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

clfs = [DecisionTreeClassifier(),RandomForestClassifier(),
        SVC(kernel='linear'),LogisticRegression(),
        GaussianNB(),KNeighborsClassifier(),
        SVC(),]

fig, axs = plt.subplots(3,3,figsize=(15,10))

for clf,ax in zip(clfs,axs.flatten()):
    clf.fit(X1,y1)
    ax.scatter(X1[:,0],X1[:,1],c=y1)
    DecisionBoundaryDisplay.from_estimator(clf, X1,
                                           ax=ax,alpha=0.5)
    ax.set_title(clf.__class__.__name__)
    ax.axis('off')
fig.show()

# Comparación de las fronteras de decisión: Ejemplo 2

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

X, y = make_blobs(n_samples=100, centers=2, random_state=30) # 27

plt.figure()
plt.scatter(X[:,0],X[:,1],c=y)
plt.show()

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay

clfs = [DecisionTreeClassifier(),RandomForestClassifier(),
        SVC(kernel='linear'),LogisticRegression(),
        GaussianNB(),KNeighborsClassifier(),SVC()]

fig, axs = plt.subplots(3,3,figsize=(15,10))

for clf,ax in zip(clfs,axs.flatten()):
    clf.fit(X,y)
    ax.scatter(X[:,0],X[:,1],c=y)
    DecisionBoundaryDisplay.from_estimator(clf, X,
                                           ax=ax,alpha=0.5)
    ax.set_title(clf.__class__.__name__)
    ax.axis('off')
fig.show()